<a href="https://colab.research.google.com/github/ishtrain07/Heart-Classifier_LALE/blob/master/LLM_News_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install requests beautifulsoup4 newspaper3k transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.

In [4]:
import newspaper
from transformers import pipeline
from newspaper import Article

# Initialize the summarizer
summarizer = pipeline("summarization", model="t5-base")


# #Specify the model to use

# model_name = "facebook/bart-large-cnn"


# # Load the summarization pipeline with the specified model
# summarizer = pipeline("summarization", model=model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [5]:
# Function to fetch and parse a single article
def fetch_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return {
            "title": article.title,
            "text": article.text,
            "url": url
        }
    except Exception as e:
        print(f"Failed to fetch article: {e}")
        return None

# Function to summarize an article
def summarize_article(article_text, max_length_ratio=0.3, min_length_ratio=0.1, do_sample=False):
    try:
        input_length = len(article_text.split())
        # Dynamically set max_length and min_length based on input length to allow for more dynamic summaries
        max_length = max(int(input_length * max_length_ratio), 50)
        min_length = max(int(input_length * min_length_ratio), 25)

        # Summarize the article
        summary = summarizer(article_text, max_length=max_length, min_length=min_length, do_sample=do_sample)

        # Log the full summary output for debugging
        print("Full summary output:", summary)

        # Ensure the output is in the expected format
        if summary and isinstance(summary, list) and len(summary) > 0 and 'summary_text' in summary[0]:
            return summary[0]['summary_text']
        else:
            print("Unexpected summarizer output format.")
            return "Summary unavailable."
    except IndexError as e:
        print(f"Failed to summarize article: index out of range - {e}")
        return "Summary unavailable."
    except Exception as e:
        print(f"Failed to summarize article: {e}")
        return "Summary unavailable."

# Main function
def main():
    print("Welcome to the Article Summarizer!")
    url = input("Please enter the exact URL of the news article to summarize: ").strip()
    if not url:
        print("URL cannot be empty. Exiting.")
        return

    article = fetch_article(url)
    if article:
        print(f"Title: {article['title']}")
        if article['text']:
            original_length = len(article['text'].split())
            print(f"Original Length: {original_length} words")
            summary = summarize_article(article['text'])  # Pass the article text to summarize_article
            summary_length = len(summary.split())
            print(f"Summary Length: {summary_length} words")
            print(f"Summary: {summary}\n")
        else:
            print("No text available for summarization.\n")
    else:
        print("Failed to fetch the article.")



if __name__ == "__main__":
    main()


Welcome to the Article Summarizer!
Please enter the exact URL of the news article to summarize: https://edition.cnn.com/2024/07/22/science/dark-oxygen-discovery-deep-sea-mining/index.html
Title: Scientists discover ‘dark’ oxygen being produced more than 13,000 feet below the ocean surface
Original Length: 1917 words
Full summary output: [{'summary_text': 'scientists from the university of hawaii and the u.s. have found that deep-sea nodules are producing dark oxygen . the bubbles are being released as the water has warmed up during its ascent to the surface . this could help unravel the origins of life on the ocean floor . but critics say mining could irrevocably damage the ocean\'s pristine ecosystems . they say it could also disrupt the way carbon is stored in the ocean, contributing to the climate crisis .  .. .  gra n   .       n  . in  s  - ., . and  gragra .-  e  ,  t   "  '}]
Summary Length: 101 words
Summary: scientists from the university of hawaii and the u.s. have found that